In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd

def generate_granular_hrv_data(n_intervals=1000, samples_per_interval=60):
    """
    Generate synthetic granular HRV data.

    :param n_intervals: Number of one-minute intervals to simulate.
    :param samples_per_interval: Number of RR interval samples per one-minute interval.
    :return: DataFrame with synthetic granular HRV data.
    """
    np.random.seed(42)  # For reproducibility

    # Generate synthetic RR intervals (in ms)
    rr_intervals = np.random.normal(850, 100, n_intervals * samples_per_interval)  # Example parameters

    # Assign each RR interval to an interval ID
    interval_ids = np.repeat(range(n_intervals), samples_per_interval)

    # Create DataFrame
    granular_hrv_data = pd.DataFrame({
        'Interval_Id': interval_ids,
        'RR_Interval': rr_intervals
    })

    return granular_hrv_data

# Generate synthetic granular HRV data
synthetic_granular_hrv = generate_granular_hrv_data()
synthetic_granular_hrv.head()

,Interval_Id,RR_Interval
0,0,899.671415
1,0,836.173570
2,0,914.768854
3,0,1002.302986
4,0,826.584663


In [2]:
def aggregate_hrv_metrics(granular_hrv_data):
    """
    Aggregate granular HRV data into HRV metrics per interval.

    :param granular_hrv_data: DataFrame with granular HRV data.
    :return: DataFrame with aggregated HRV metrics per interval.
    """
    aggregated_data = granular_hrv_data.groupby('Interval_Id').apply(lambda x: pd.Series({
        'MEAN_RR': x['RR_Interval'].mean(),
        'MEDIAN_RR': x['RR_Interval'].median(),
        'SDRR': x['RR_Interval'].std(),
        'RMSSD': np.sqrt(np.mean(np.diff(x['RR_Interval'])**2)),
        'SDSD': np.std(np.diff(x['RR_Interval'])),
        'SDRR_RMSSD': x['RR_Interval'].std() / np.sqrt(np.mean(np.diff(x['RR_Interval'])**2)) if np.sqrt(np.mean(np.diff(x['RR_Interval'])**2)) != 0 else 0,
        'HR': 60000 / x['RR_Interval'].mean() if x['RR_Interval'].mean() != 0 else 0
        # Add other HRV metrics as needed
    })).reset_index()

    return aggregated_data

# Aggregate HRV metrics from synthetic granular HRV data
synthetic_aggregated_hrv = aggregate_hrv_metrics(synthetic_granular_hrv)
synthetic_aggregated_hrv.head(20)

,Interval_Id,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR
0,0,834.534532,827.004337,90.852112,124.315676,124.313027,0.730818,71.896366
1,1,849.631657,846.938718,94.336318,135.716473,135.700431,0.695099,70.618838
2,2,859.191425,872.985682,99.610318,151.761315,151.726087,0.656362,69.833099
3,3,855.681427,866.345302,103.636113,147.393815,147.393314,0.703124,70.119554
4,4,848.186679,847.535385,104.444083,159.785293,159.767111,0.653653,70.739144
5,5,853.766373,848.462005,76.151320,102.795262,102.795262,0.740806,70.276837
6,6,862.846891,860.667537,99.498536,141.009494,141.007414,0.705616,69.537250
7,7,849.459841,842.410043,113.838523,141.680709,141.671729,0.803486,70.633121
8,8,823.265389,806.415954,90.009953,131.318723,131.316519,0.685431,72.880508
9,9,849.916515,857.907801,96.002299,135.556031,135.553805,0.708211,70.595169


In [5]:
import pickle

folder_model_path = '/content/drive/My Drive/Colab Notebooks/Ch4_HR_HRV_Generation/data-hrv-models/'
model_file_name = 'model_rf_fset_2.sav'
# load the previously saved model from a file
model = pickle.load(open(folder_model_path + model_file_name, 'rb'))

In [6]:
X_test = synthetic_aggregated_hrv[['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD', 'SDRR_RMSSD','HR']]  # Adjust this based on your actual feature set

# Predict stress or no-stress states
predicted_classes = model.predict(X_test)

In [7]:
unique, counts = np.unique(predicted_classes, return_counts=True)
predicted_counts = dict(zip(unique, counts))
print(predicted_counts)

{0: 1000}
